# Main

## Prep

In [1]:
# File Manipulation

import os # For working with Operating System
import sys # System arguments
from dotenv import load_dotenv # Loading .env info

# Web

import requests # Accessing the Web

# Time

import datetime as dt # Working with dates/times
import pytz # Timezones
import time # For Sleeping

# Database 

import psycopg2
from psycopg2 import sql

# Data Manipulation

import numpy as np
import geopandas as gpd
import pandas as pd

### Load Functions

In [11]:
script_path = os.path.join('..', '..', 'Scripts', 'python')

# Function definition - Please see Scripts/python/*
exec(open(os.path.join(script_path, 'Get_spikes_df.py')).read())
exec(open(os.path.join(script_path, 'Create_messages.py')).read())
exec(open(os.path.join(script_path, 'twilio_functions.py')).read())
exec(open(os.path.join(script_path, 'Update_Alerts.py')).read())
exec(open(os.path.join(script_path, 'Send_Alerts.py')).read())

### Global Variables

In [5]:
load_dotenv() # Load .env file

## API Keys

purpleAir_api = os.getenv('PURPLEAIR_API_TOKEN') # PurpleAir API Read Key

redCap_token_signUp = os.getenv('REDCAP_TOKEN_SIGNUP') # Survey Token
redCap_token_report = os.getenv('REDCAP_TOKEN_REPORT') # Report Token

## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))  

## Twilio Information

TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')

# Other Constants from System Arguments

spike_threshold = 35 # Value which defines an AQ_Spike (Micgrograms per meter cubed)

timestep = 10 # Sleep time in between updates (in Minutes)

# When to stop the program? (datetime)
days_to_run = 7 # How many days will we run this?
stoptime = dt.datetime.now() + dt.timedelta(days=days_to_run)

# Waking hours
too_late_hr = 21 # 9pm
too_early_hr = 8 # 8am

# Report URL

base_report_url = 'https://redcap.ahc.umn.edu/surveys/?s=LN3HHDCJXYCKFCLE'

## While Loop

In [12]:
### The Script

# Initialize storage for reports_for_day

reports_for_day = 0

while True:

    now = dt.datetime.now() # The current time

    print(now)

    if stoptime < now: # Check if we've hit stoptime
        break
   
   # ~~~~~~~~~~~~~~~~~~~~~
   
   # if new day: # NOT DONE
   #     # Initialize reports_for_day
   #     reports_for_day = 0
   
   # ~~~~~~~~~~~~~~~~~~~~~
   
   # Query PurpleAir

    #  Get the sensor_ids from sensors in our database

    sensor_ids = get_sensor_ids(pg_connection_dict) # In Get_Spikes_df.py

    # Query PurpleAir for Spikes

    spikes_df, runtime, flagged_sensor_ids = Get_spikes_df(purpleAir_api, sensor_ids, spike_threshold) # In Get_Spikes_df.py

    # Sort the spiked sensors into new, ongoing, ended spiked sensors, and not spiked sensors

    new_spike_sensors, ongoing_spike_sensors, ended_spike_sensors, not_spiked_sensors = sort_sensors_for_updates(spikes_df, sensor_ids, flagged_sensor_ids, pg_connection_dict) # In Update_Alerts.py

    # Initialize message/record_id storage
    
    record_ids_to_text = []
    messages = []
    
    # ~~~~~~~~~~~~~~~~~~~~~
    
    # NEW Spikes
    
    if len(new_spike_sensors) > 0:

        new_spikes_df = spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)] 
    
        for index, row in new_spikes_df.iterrows():
    
            # 1) Add to active alerts
        
            newest_alert_index = add_to_active_alerts(row, pg_connection_dict,
                                 runtime.strftime('%Y-%m-%d %H:%M:%S') # When we ran the PurpleAir Query
                                ) # In Update_Alerts.py
            
            # 2) Query users ST_Dwithin 1000 meters & subscribed = TRUE
            
            record_ids_nearby = Users_nearby_sensor(pg_connection_dict, row.sensor_index, 1000) # in Send_Alerts.py
            
            if len(record_ids_nearby) > 0:

                if (now.hour < too_late_hr) & (now.hour > too_early_hr): # Waking Hours
            
                    # a) Query users from record_ids_nearby if both active_alerts and cached_alerts are empty
                    record_ids_new_alerts = Users_to_message_new_alert(pg_connection_dict, record_ids_nearby) # in Send_Alerts.py & .ipynb 
                    
                    # Compose Messages & concat to messages/record_id_to_text   
                    
                    # Add to message/record_id storage for future messaging
                    record_ids_to_text += record_ids_new_alerts
                    messages += [new_alert_message(sensor_id)]*len(record_ids_new_alerts) # in Compose_Messages.py

                # b) Add newest_alert_index to record_ids_nearby's Active Alerts
    
                update_users_active_alerts(record_ids_nearby, newest_alert_index, pg_connection_dict) # in Update_Alerts.py & .ipynb

    # ~~~~~~~~~~~~~~~~~~~~~

    # ONGOING spikes

    if len(ongoing_spike_sensors) > 0:

        ongoing_spikes_df = spikes_df[spikes_df.sensor_index.isin(ongoing_spike_sensors)]

        for _, spike in ongoing_spikes_df.iterrows():

            # 1) Update the maximum reading
    
            update_max_reading(spike, pg_connection_dict) # In Update_Alerts.py
            
            # 2) Merge/Cluster alerts? 
            # NOT DONE - FAR FUTURE TO DO

    # ~~~~~~~~~~~~~~~~~~~~~

    # ENDED spikes

    if len(ended_spike_sensors) > 0:

        # 1) Add alert to archive
    
        add_to_archived_alerts(not_spiked_sensors, pg_connection_dict) # In Update_Alerts.py

        # 2) Remove from Active Alerts
        
        ended_alert_indices = remove_active_alerts(not_spiked_sensors, pg_connection_dict) # # A list from Update_Alerts.py

        # 3) Transfer these alerts from "Sign Up Information" active_alerts to "Sign Up Information" cached_alerts 
        
        cache_alerts(ended_alert_indices, pg_connection_dict) # in Update_Alerts.py & .ipynb

        # 4) Query for people to text (subscribed = TRUE and active_alerts is empty and cached_alerts not empty and cached_alerts is > 10 minutes old - ie. ended_alert_indices intersect cached_alerts is empty) 
        
        record_ids_end_alert_message = Users_to_message_end_alert(pg_connection_dict, ended_alert_indices) # in Send_Alerts.py & .ipynb
                
        # 5) If #4 has elements: for each element (user) in #4
        
        if len(record_ids_end_alert_message) > 0:
        
            for record_id in record_ids_end_alert_message:
                
                # a) Initialize report - generate unique report_id, log cached_alerts and use to find start_time/max reading/duration/sensor_indices
                
                report_id = str(reports_for_day).zfill(5) + '-' + now.strftime('%m%d%y') # XXXXX-MMDDYY
                    
                duration_minutes, max_reading = initialize_report(record_id, reports_for_day, pg_connection_dict) # in Send_Alerts.py & .ipynb
                
                reports_for_day += 1 
                
                if (now.hour < too_late_hr) & (now.hour > too_early_hr): # Waking hours

                    # b) Compose message telling user it's over w/ unique report option & concat to messages/record_ids_to_text
                    
                    record_ids_to_text += [record_id]
                    messages += [end_alert_message(duration_minutes, max_reading, report_id, base_report_url)]

                # c) Clear the user's cached_alerts 
                # - NOT DONE - do in Update_Alerts.py & .ipynb
                
    # ~~~~~~~~~~~~~~~~~~~~~           
    
    # Send all messages - NOT DONE do in Send_Alerts.py & .ipynb
    
    # ~~~~~~~~~~~~~~~~~~~~~

    # SLEEP between updates

    when_to_awake = now + dt.timedelta(minutes=timestep) 

    sleep_seconds = (when_to_awake - dt.datetime.now()).seconds # - it takes about 3 seconds to run through everything

    time.sleep(sleep_seconds) # Sleep


print("Terminating Program")

2023-11-06 15:47:23.714773


KeyboardInterrupt: 

In [14]:
record_ids_to_text

[1, 2]

In [13]:
messages

['Alert Over\nDuration: 16738 minutes \nMax value: 79.0 ug/m3\n\nReport here - https://redcap.ahc.umn.edu/surveys/?s=LN3HHDCJXYCKFCLE&report_id=00000-110623',
 'Alert Over\nDuration: 1384 minutes \nMax value: 35.0 ug/m3\n\nReport here - https://redcap.ahc.umn.edu/surveys/?s=LN3HHDCJXYCKFCLE&report_id=00001-110623']